Data Prep

In [2]:
from tensorflow.python.ops.rnn_cell_impl import *

In [3]:
#https://arxiv.org/pdf/1603.09727.pdf
import pandas as pd
import numpy as np, os, shutil
import string
train = pd.read_csv('../input/en_train.csv')
len(train)
x = train.groupby('sentence_id')['before'].apply(lambda x : x.str.cat(sep = ' '))
len(x)
y = train.groupby('sentence_id')['after'].apply(lambda x : x.str.cat(sep = ' '))
split = int(0.9*len(x))
split
x, y
train_x, valid_x = x[:split], x[split:]
train_y, valid_y = y[:split], y[split:]

train_x
#shutil.rmtree('./char')
os.mkdir('./char')
train_x.to_csv('./char/train.x.txt', index = False, header = False)
valid_x.to_csv('./char/valid.x.txt', index = False, header = False)
train_y.to_csv('./char/train.y.txt', index = False, header = False)
valid_y.to_csv('./char/valid.y.txt', index = False, header = False)
os.listdir('./char')

NLC Data

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
import gzip
import os
import re
import tarfile
from six.moves import urllib
from tensorflow.python.platform import gfile

# Special vocabulary symbols - we always put them at the start.
_PAD = b"<pad>"
_SOS = b"<sos>"
_EOS = b"<eos>"
_UNK = b"<unk>"
_START_VOCAB = [_PAD, _SOS, _EOS, _UNK]

PAD_ID = 0
SOS_ID = 1
EOS_ID = 2
UNK_ID = 3

# Regular expressions used to tokenize.
_WORD_SPLIT = re.compile(b"([.,!?\"':;)(])")
_DIGIT_RE = re.compile(br"\d")

#./train.x.txt ==> train['before'] groupby sentences train.y.txt group by after
def get_nlc_train_set(directory):
  train_path = os.path.join(directory, "train")
  print (train_path + ".x.txt")
  print (train_path + ".y.txt")
  return train_path


def get_nlc_dev_set(directory):
  dev_name = "valid"
  dev_path = os.path.join(directory, dev_name)
  return dev_path


def basic_tokenizer(sentence):
  words = []
  for space_separated_fragment in sentence.strip().split():
    words.extend(re.split(_WORD_SPLIT, space_separated_fragment))
  return [w for w in words if w]

def char_tokenizer(sentence):
  return list(sentence.strip())

def bpe_tokenizer(sentence):
  tokens = sentence.strip().split()
  tokens = [w + "</w>" if not w.endswith("@@") else w for w in tokens]
  tokens = [w.replace("@@", "") for w in tokens]
  return tokens

def remove_nonascii(text):
  return re.sub(r'[^\x00-\x7F]', '', text.decode("utf-8") ).encode('utf-8')

def create_vocabulary(vocabulary_path, data_paths, max_vocabulary_size,
                      tokenizer=None, normalize_digits=False):
  if not gfile.Exists(vocabulary_path):
    print("Creating vocabulary %s from data %s" % (vocabulary_path, str(data_paths)))
    vocab = {}
    for path in data_paths:
      with gfile.GFile(path, mode="rb") as f:
        counter = 0
        for line in f:
          counter += 1
          if counter % 100000 == 0:
            print("  processing line %d" % counter)
          # Remove non-ASCII characters
          line = remove_nonascii(line)
          tokens = tokenizer(line) if tokenizer else basic_tokenizer(line)
          for w in tokens:
            word = re.sub(_DIGIT_RE, b"0", w) if normalize_digits else w
            if word in vocab:
              vocab[word] += 1
            else:
              vocab[word] = 1
    vocab_list = _START_VOCAB + sorted(vocab, key=vocab.get, reverse=True)
    print("Vocabulary size: %d" % len(vocab_list))
    if len(vocab_list) > max_vocabulary_size:
      vocab_list = vocab_list[:max_vocabulary_size]
    with gfile.GFile(vocabulary_path, mode="wb") as vocab_file:
      for w in vocab_list:
        vocab_file.write(str(w).encode('utf-8')+ b"\n")


def initialize_vocabulary(vocabulary_path, bpe=False):
  if gfile.Exists(vocabulary_path):
    rev_vocab = []
    with gfile.GFile(vocabulary_path, mode="rb") as f:
      rev_vocab.extend(f.readlines())
    rev_vocab = [line.strip(b'\n') for line in rev_vocab]
    # Call ''.join below since BPE outputs split pairs with spaces
    if bpe:
      vocab = dict([(''.join(x.split(' ')), y) for (y, x) in enumerate(rev_vocab)])
    else:
      vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
    return vocab, rev_vocab
  else:
    raise ValueError("Vocabulary file %s not found.", vocabulary_path)


def sentence_to_token_ids(sentence, vocabulary,
                          tokenizer=None, normalize_digits=False):
  if tokenizer:
    words = tokenizer(sentence)
  else:
    words = basic_tokenizer(sentence)
  if not normalize_digits:
    return [vocabulary.get(w, UNK_ID) for w in words]
  # Normalize digits by 0 before looking words up in the vocabulary.
  return [vocabulary.get(re.sub(_DIGIT_RE, b"0", w), UNK_ID) for w in words]


def data_to_token_ids(data_path, target_path, vocabulary_path,
                      tokenizer=None, normalize_digits=False):
  if not gfile.Exists(target_path):
    print("Tokenizing data in %s" % data_path)
    vocab, _ = initialize_vocabulary(vocabulary_path, bpe=(tokenizer==bpe_tokenizer))
    with gfile.GFile(data_path, mode="rb") as data_file:
      with gfile.GFile(target_path, mode="w") as tokens_file:
        counter = 0
        for line in data_file:
          counter += 1
          if counter % 100000 == 0:
            print("  tokenizing line %d" % counter)
          line = remove_nonascii(line)
          token_ids = sentence_to_token_ids(line, vocab, tokenizer,
                                            normalize_digits)
          tokens_file.write(" ".join([str(tok) for tok in token_ids]) + "\n")


def prepare_nlc_data(data_dir, max_vocabulary_size, tokenizer=char_tokenizer, other_dev_path=None):
  # Get nlc data to the specified directory.
  train_path = get_nlc_train_set(data_dir)
  dev_path = get_nlc_dev_set(data_dir)
  vocab_path = os.path.join(data_dir, "vocab.dat")
  if tokenizer != bpe_tokenizer:
    create_vocabulary(vocab_path, [train_path + ".y.txt", train_path + ".x.txt"],
                      max_vocabulary_size, tokenizer)

  # Create token ids for the training data.
  y_train_ids_path = train_path + ".ids.y"
  x_train_ids_path = train_path + ".ids.x"
  data_to_token_ids(train_path + ".y.txt", y_train_ids_path, vocab_path, tokenizer)
  data_to_token_ids(train_path + ".x.txt", x_train_ids_path, vocab_path, tokenizer)

  # Create token ids for the development data.
  y_dev_ids_path = dev_path + ".ids.y"
  x_dev_ids_path = dev_path + ".ids.x"
  data_to_token_ids(dev_path + ".y.txt", y_dev_ids_path, vocab_path, tokenizer)
  data_to_token_ids(dev_path + ".x.txt", x_dev_ids_path, vocab_path, tokenizer)

  return (x_train_ids_path, y_train_ids_path,
          x_dev_ids_path, y_dev_ids_path, vocab_path)

NLC Model

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import rnn
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh
from tensorflow.python.ops.rnn_cell_impl import *

def get_optimizer(opt):
  if opt == "adam":
    optfn = tf.train.AdamOptimizer
  elif opt == "sgd":
    optfn = tf.train.GradientDescentOptimizer
  else:
    assert(False)
  return optfn

class GRUCellAttn(rnn_cell.GRUCell):
  def __init__(self, num_units, encoder_output, scope=None):
    self.hs = encoder_output
    with vs.variable_scope(scope or type(self).__name__):
      with vs.variable_scope("Attn1") as scope:
        hs2d = tf.reshape(self.hs, [-1, num_units])
        phi_hs2d = tanh(_linear(hs2d, num_units, True, 1.0))
        self.phi_hs = tf.reshape(phi_hs2d, tf.shape(self.hs))
    super(GRUCellAttn, self).__init__(num_units)

  def __call__(self, inputs, state, scope=None):
    gru_out, gru_state = super(GRUCellAttn, self).__call__(inputs, state, scope)
    with vs.variable_scope(scope or type(self).__name__):
      with vs.variable_scope("Attn2"):
        gamma_h = tanh(_linear(gru_out, self._num_units, True, 1.0))
      weights = tf.reduce_sum(self.phi_hs * gamma_h, reduction_indices=2, keep_dims=True)
      weights = tf.exp(weights - tf.reduce_max(weights, reduction_indices=0, keep_dims=True))
      weights = weights / (1e-6 + tf.reduce_sum(weights, reduction_indices=0, keep_dims=True))
      context = tf.reduce_sum(self.hs * weights, reduction_indices=0)
      with vs.variable_scope("AttnConcat"):
        out = tf.nn.relu(_linear([context, gru_out], self._num_units, True, 1.0))
      self.attn_map = tf.squeeze(tf.slice(weights, [0, 0, 0], [-1, -1, 1]))
      return (out, out)


class NLCModel(object):
  def __init__(self, vocab_size, size, num_layers, max_gradient_norm, batch_size, learning_rate,
               learning_rate_decay_factor, dropout, forward_only=False, optimizer="adam"):

    self.size = size
    self.vocab_size = vocab_size
    self.batch_size = batch_size
    self.num_layers = num_layers
    self.keep_prob_config = 1.0 - dropout
    self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
    self.learning_rate_decay_op = self.learning_rate.assign(self.learning_rate * learning_rate_decay_factor)
    self.global_step = tf.Variable(0, trainable=False)

    self.keep_prob = tf.placeholder(tf.float32)
    self.source_tokens = tf.placeholder(tf.int32, shape=[None, None])
    self.target_tokens = tf.placeholder(tf.int32, shape=[None, None])
    self.source_mask = tf.placeholder(tf.int32, shape=[None, None])
    self.target_mask = tf.placeholder(tf.int32, shape=[None, None])
    self.beam_size = tf.placeholder(tf.int32)
    self.target_length = tf.reduce_sum(self.target_mask, reduction_indices=0)

    self.decoder_state_input, self.decoder_state_output = [], []
    for i in xrange(num_layers):
      self.decoder_state_input.append(tf.placeholder(tf.float32, shape=[None, size]))

    with tf.variable_scope("NLC", initializer=tf.uniform_unit_scaling_initializer(1.0)):
      self.setup_embeddings() 
      self.setup_encoder()
      self.setup_decoder()
      self.setup_loss()

      self.setup_beam()

    params = tf.trainable_variables()
    if not forward_only:
      opt = get_optimizer(optimizer)(self.learning_rate)

      gradients = tf.gradients(self.losses, params)
      clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
#      self.gradient_norm = tf.global_norm(clipped_gradients)
      self.gradient_norm = tf.global_norm(gradients)
      self.param_norm = tf.global_norm(params)
      self.updates = opt.apply_gradients(
        zip(clipped_gradients, params), global_step=self.global_step)

    self.saver = tf.train.Saver(tf.all_variables(), max_to_keep=0)

  def setup_embeddings(self):
    with vs.variable_scope("embeddings") as scope:
      self.L_enc = tf.get_variable("L_enc", [self.vocab_size, self.size])
      self.L_dec = tf.get_variable("L_dec", [self.vocab_size, self.size])
      self.encoder_inputs = embedding_ops.embedding_lookup(self.L_enc, self.source_tokens)
      self.decoder_inputs = embedding_ops.embedding_lookup(self.L_dec, self.target_tokens)

  def setup_encoder(self):
    self.encoder_cell = rnn_cell.GRUCell(self.size)
    with vs.variable_scope("PryamidEncoder"):
      inp = self.encoder_inputs
      mask = self.source_mask
      out = None
      for i in xrange(self.num_layers):
        with vs.variable_scope("EncoderCell%d" % i) as scope:
          srclen = tf.reduce_sum(mask, reduction_indices=0)
          out, _ = self.bidirectional_rnn(self.encoder_cell, inp, srclen, scope=scope)
          dropin, mask = self.downscale(out, mask)
          inp = self.dropout(dropin)
      self.encoder_output = out

  def setup_decoder(self):
    if self.num_layers > 1:
      self.decoder_cell = rnn_cell.GRUCell(self.size)
    self.attn_cell = GRUCellAttn(self.size, self.encoder_output, scope="DecoderAttnCell")

    with vs.variable_scope("Decoder"):
      inp = self.decoder_inputs
      for i in xrange(self.num_layers - 1):
        with vs.variable_scope("DecoderCell%d" % i) as scope:
          out, state_output = rnn.dynamic_rnn(self.decoder_cell, inp, time_major=True,
                                              dtype=dtypes.float32, sequence_length=self.target_length,
                                              scope=scope, initial_state=self.decoder_state_input[i])
          inp = self.dropout(out)
          self.decoder_state_output.append(state_output)

      with vs.variable_scope("DecoderAttnCell") as scope:
        out, state_output = rnn.dynamic_rnn(self.attn_cell, inp, time_major=True,
                                            dtype=dtypes.float32, sequence_length=self.target_length,
                                            scope=scope, initial_state=self.decoder_state_input[i+1])
        self.decoder_output = self.dropout(out)
        self.decoder_state_output.append(state_output)

  def decoder_graph(self, decoder_inputs, decoder_state_input):
    decoder_output, decoder_state_output = None, []
    inp = decoder_inputs

    with vs.variable_scope("Decoder", reuse=True):
      for i in xrange(self.num_layers - 1):
        with vs.variable_scope("DecoderCell%d" % i) as scope:
          inp, state_output = self.decoder_cell(inp, decoder_state_input[i])
          decoder_state_output.append(state_output)

      with vs.variable_scope("DecoderAttnCell") as scope:
        decoder_output, state_output = self.attn_cell(inp, decoder_state_input[i+1])
        decoder_state_output.append(state_output)

    return decoder_output, decoder_state_output

  def setup_beam(self):
    time_0 = tf.constant(0)
    beam_seqs_0 = tf.constant([[SOS_ID]])
    beam_probs_0 = tf.constant([0.])

    cand_seqs_0 = tf.constant([[EOS_ID]])
    cand_probs_0 = tf.constant([-3e38])

    state_0 = tf.zeros([1, self.size])
    states_0 = [state_0] * self.num_layers

    def beam_cond(time, beam_probs, beam_seqs, cand_probs, cand_seqs, *states):
      return tf.reduce_max(beam_probs) >= tf.reduce_min(cand_probs)

    def beam_step(time, beam_probs, beam_seqs, cand_probs, cand_seqs, *states):
      batch_size = tf.shape(beam_probs)[0]
      inputs = tf.reshape(tf.slice(beam_seqs, [0, time], [batch_size, 1]), [batch_size])
      decoder_input = embedding_ops.embedding_lookup(self.L_dec, inputs)
      decoder_output, state_output = self.decoder_graph(decoder_input, states)

      with vs.variable_scope("Logistic", reuse=True):
        do2d = tf.reshape(decoder_output, [-1, self.size])
        logits2d = _linear(do2d, self.vocab_size, True, 1.0)
        logprobs2d = tf.nn.log_softmax(logits2d)

      total_probs = logprobs2d + tf.reshape(beam_probs, [-1, 1])
      total_probs_noEOS = tf.concat(1, [tf.slice(total_probs, [0, 0], [batch_size, EOS_ID]),
                                        tf.tile([[-3e38]], [batch_size, 1]),
                                        tf.slice(total_probs, [0, EOS_ID + 1],
                                                 [batch_size, self.vocab_size - EOS_ID - 1])])

      flat_total_probs = tf.reshape(total_probs_noEOS, [-1])
      beam_k = tf.minimum(tf.size(flat_total_probs), self.beam_size)
      next_beam_probs, top_indices = tf.nn.top_k(flat_total_probs, k=beam_k)

      next_bases = tf.floordiv(top_indices, self.vocab_size)
      next_mods = tf.mod(top_indices, self.vocab_size)

      next_states = [tf.gather(state, next_bases) for state in state_output]
      next_beam_seqs = tf.concat(1, [tf.gather(beam_seqs, next_bases),
                                     tf.reshape(next_mods, [-1, 1])])

      cand_seqs_pad = tf.pad(cand_seqs, [[0, 0], [0, 1]])
      beam_seqs_EOS = tf.pad(beam_seqs, [[0, 0], [0, 1]])
      new_cand_seqs = tf.concat(0, [cand_seqs_pad, beam_seqs_EOS])
      EOS_probs = tf.slice(total_probs, [0, EOS_ID], [batch_size, 1])
      new_cand_probs = tf.concat(0, [cand_probs, tf.reshape(EOS_probs, [-1])])

      cand_k = tf.minimum(tf.size(new_cand_probs), self.beam_size)
      next_cand_probs, next_cand_indices = tf.nn.top_k(new_cand_probs, k=cand_k)
      next_cand_seqs = tf.gather(new_cand_seqs, next_cand_indices)

      return [time + 1, next_beam_probs, next_beam_seqs, next_cand_probs, next_cand_seqs] + next_states

    var_shape = []
    var_shape.append((time_0, time_0.get_shape()))
    var_shape.append((beam_probs_0, tf.TensorShape([None,])))
    var_shape.append((beam_seqs_0, tf.TensorShape([None, None])))
    var_shape.append((cand_probs_0, tf.TensorShape([None,])))
    var_shape.append((cand_seqs_0, tf.TensorShape([None, None])))
    var_shape.extend([(state_0, tf.TensorShape([None, self.size])) for state_0 in states_0])
    loop_vars, loop_var_shapes = zip(* var_shape)
    ret_vars = tf.while_loop(cond=beam_cond, body=beam_step, loop_vars=loop_vars, shape_invariants=loop_var_shapes, back_prop=False)
#    time, beam_probs, beam_seqs, cand_probs, cand_seqs, _ = ret_vars
    cand_seqs = ret_vars[4]
    cand_probs = ret_vars[3]
    self.beam_output = cand_seqs
    self.beam_scores = cand_probs

  def setup_loss(self):
    with vs.variable_scope("Logistic"):
      doshape = tf.shape(self.decoder_output)
      T, batch_size = doshape[0], doshape[1]
      do2d = tf.reshape(self.decoder_output, [-1, self.size])
      logits2d = _linear(do2d, self.vocab_size, True, 1.0)
      outputs2d = tf.nn.log_softmax(logits2d)
      self.outputs = tf.reshape(outputs2d, tf.pack([T, batch_size, self.vocab_size]))

      targets_no_GO = tf.slice(self.target_tokens, [1, 0], [-1, -1])
      masks_no_GO = tf.slice(self.target_mask, [1, 0], [-1, -1])
      # easier to pad target/mask than to split decoder input since tensorflow does not support negative indexing
      labels1d = tf.reshape(tf.pad(targets_no_GO, [[0, 1], [0, 0]]), [-1])
      mask1d = tf.reshape(tf.pad(masks_no_GO, [[0, 1], [0, 0]]), [-1])
      losses1d = tf.nn.sparse_softmax_cross_entropy_with_logits(logits2d, labels1d) * tf.to_float(mask1d)
      losses2d = tf.reshape(losses1d, tf.pack([T, batch_size]))
      self.losses = tf.reduce_sum(losses2d) / tf.to_float(batch_size)

  def dropout(self, inp):
    return tf.nn.dropout(inp, self.keep_prob)

  def downscale(self, inp, mask):
    return inp, mask

    with vs.variable_scope("Downscale"):
      inshape = tf.shape(inp)
      T, batch_size, dim = inshape[0], inshape[1], inshape[2]
      inp2d = tf.reshape(tf.transpose(inp, perm=[1, 0, 2]), [-1, 2 * self.size])
      out2d = _linear(inp2d, self.size, True, 1.0)
      out3d = tf.reshape(out2d, tf.pack((batch_size, tf.to_int32(T/2), dim)))
      out3d = tf.transpose(out3d, perm=[1, 0, 2])
      out3d.set_shape([None, None, self.size])
      out = tanh(out3d)

      mask = tf.transpose(mask)
      mask = tf.reshape(mask, [-1, 2])
      mask = tf.cast(mask, tf.bool)
      mask = tf.reduce_any(mask, reduction_indices=1)
      mask = tf.to_int32(mask)
      mask = tf.reshape(mask, tf.pack([batch_size, -1]))
      mask = tf.transpose(mask)
    return out, mask

  def bidirectional_rnn(self, cell, inputs, lengths, scope=None):
    name = scope.name or "BiRNN"
    # Forward direction
    with vs.variable_scope(name + "_FW") as fw_scope:
      output_fw, output_state_fw = rnn.dynamic_rnn(cell, inputs, time_major=True, dtype=dtypes.float32,
                                                   sequence_length=lengths, scope=fw_scope)
    # Backward direction
    inputs_bw = tf.reverse_sequence(inputs, tf.to_int64(lengths), seq_dim=0, batch_dim=1)
    with vs.variable_scope(name + "_BW") as bw_scope:
      output_bw, output_state_bw = rnn.dynamic_rnn(cell, inputs_bw, time_major=True, dtype=dtypes.float32,
                                                   sequence_length=lengths, scope=bw_scope)

    output_bw = tf.reverse_sequence(output_bw, tf.to_int64(lengths), seq_dim=0, batch_dim=1)

    outputs = output_fw + output_bw
    output_state = output_state_fw + output_state_bw

    return (outputs, output_state)

  def set_default_decoder_state_input(self, input_feed, batch_size):
    default_value = np.zeros([batch_size, self.size])
    for i in xrange(self.num_layers):
      input_feed[self.decoder_state_input[i]] = default_value

  def train(self, session, source_tokens, source_mask, target_tokens, target_mask):
    input_feed = {}
    input_feed[self.source_tokens] = source_tokens
    input_feed[self.target_tokens] = target_tokens
    input_feed[self.source_mask] = source_mask
    input_feed[self.target_mask] = target_mask
    input_feed[self.keep_prob] = self.keep_prob_config
    self.set_default_decoder_state_input(input_feed, target_tokens.shape[1])

    output_feed = [self.updates, self.gradient_norm, self.losses, self.param_norm]

    outputs = session.run(output_feed, input_feed)

    return outputs[1], outputs[2], outputs[3]

  def test(self, session, source_tokens, source_mask, target_tokens, target_mask):
    input_feed = {}
    input_feed[self.source_tokens] = source_tokens
    input_feed[self.target_tokens] = target_tokens
    input_feed[self.source_mask] = source_mask
    input_feed[self.target_mask] = target_mask
    input_feed[self.keep_prob] = 1.
    self.set_default_decoder_state_input(input_feed, target_tokens.shape[1])

    output_feed = [self.losses]

    outputs = session.run(output_feed, input_feed)

    return outputs[0]

  def encode(self, session, source_tokens, source_mask):
    input_feed = {}
    input_feed[self.source_tokens] = source_tokens
    input_feed[self.source_mask] = source_mask
    input_feed[self.keep_prob] = 1.

    output_feed = [self.encoder_output]

    outputs = session.run(output_feed, input_feed)

    return outputs[0]

  def decode(self, session, encoder_output, target_tokens, target_mask=None, decoder_states=None):
    input_feed = {}
    input_feed[self.encoder_output] = encoder_output
    input_feed[self.target_tokens] = target_tokens
    input_feed[self.target_mask] = target_mask if target_mask else np.ones_like(target_tokens)
    input_feed[self.keep_prob] = 1.

    if not decoder_states:
      self.set_default_decoder_state_input(input_feed, target_tokens.shape[1])
    else:
      for i in xrange(self.num_layers):
        input_feed[self.decoder_state_input[i]] = decoder_states[i]

    output_feed = [self.outputs] + self.decoder_state_output

    outputs = session.run(output_feed, input_feed)

    return outputs[0], None, outputs[1:]

  def decode_beam(self, session, encoder_output, beam_size=8):
    input_feed = {}
    input_feed[self.encoder_output] = encoder_output
    input_feed[self.keep_prob] = 1.
    input_feed[self.beam_size] = beam_size

    output_feed = [self.beam_output, self.beam_scores]

    outputs = session.run(output_feed, input_feed)

    return outputs[0], outputs[1]

Train data

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time
import random
import json

import numpy as np
from six.moves import xrange
import tensorflow as tf

import logging
logging.basicConfig(level=logging.INFO)

FLAGS = tf.app.flags.FLAGS

def tokenize(string):
  return [int(s) for s in string.split()]

def pair_iter(fnamex, fnamey, batch_size, num_layers, sort_and_shuffle=True):
  fdx, fdy = open(fnamex), open(fnamey)
  batches = []

  while True:
    if len(batches) == 0:
      refill(batches, fdx, fdy, batch_size, sort_and_shuffle=sort_and_shuffle)
    if len(batches) == 0:
      break

    x_tokens, y_tokens = batches.pop(0)
    y_tokens = add_sos_eos(y_tokens)
    x_padded, y_padded = padded(x_tokens, num_layers), padded(y_tokens, 1)

    source_tokens = np.array(x_padded).T
    source_mask = (source_tokens != PAD_ID).astype(np.int32)
    target_tokens = np.array(y_padded).T
    target_mask = (target_tokens != PAD_ID).astype(np.int32)

    yield (source_tokens, source_mask, target_tokens, target_mask)

  return

def refill(batches, fdx, fdy, batch_size, sort_and_shuffle=True):
  line_pairs = []
  linex, liney = fdx.readline(), fdy.readline()

  while linex and liney:
    x_tokens, y_tokens = tokenize(linex), tokenize(liney)

    if len(x_tokens) < FLAGS.max_seq_len and len(y_tokens) < FLAGS.max_seq_len:
      line_pairs.append((x_tokens, y_tokens))
    if len(line_pairs) == batch_size * 16:
      break
    linex, liney = fdx.readline(), fdy.readline()

  if sort_and_shuffle:
    line_pairs = sorted(line_pairs, key=lambda e: len(e[0]))

  for batch_start in xrange(0, len(line_pairs), batch_size):
    x_batch, y_batch = zip(*line_pairs[batch_start:batch_start+batch_size])
#    if len(x_batch) < batch_size:
#      break
    batches.append((x_batch, y_batch))

  if sort_and_shuffle:
    random.shuffle(batches)
  return

def add_sos_eos(tokens):
  return map(lambda token_list: [SOS_ID] + token_list + [EOS_ID], tokens)

def padded(tokens, depth):
  maxlen = max(map(lambda x: len(x), tokens))
  align = pow(2, depth - 1)
  padlen = maxlen + (align - maxlen) % align
  return map(lambda token_list: token_list + [PAD_ID] * (padlen - len(token_list)), tokens)


def get_tokenizer(flags):
  if flags.tokenizer.lower() == 'bpe':
    return bpe_tokenizer
  elif flags.tokenizer.lower() == 'char':
    return char_tokenizer
  elif flags.tokenizer.lower() == 'word':
    return basic_tokenizer
  else:
    raise
  return tokenizer


tf.app.flags.DEFINE_float("learning_rate", 0.0003, "Learning rate.")
tf.app.flags.DEFINE_float("learning_rate_decay_factor", 0.95, "Learning rate decays by this much.")
tf.app.flags.DEFINE_float("max_gradient_norm", 10.0, "Clip gradients to this norm.")
tf.app.flags.DEFINE_float("dropout", 0.15, "Fraction of units randomly dropped on non-recurrent connections.")
tf.app.flags.DEFINE_integer("batch_size", 128, "Batch size to use during training.")
tf.app.flags.DEFINE_integer("epochs", 40, "Number of epochs to train.")
tf.app.flags.DEFINE_integer("size", 400, "Size of each model layer.")
tf.app.flags.DEFINE_integer("num_layers", 3, "Number of layers in the model.")
tf.app.flags.DEFINE_integer("max_vocab_size", 40000, "Vocabulary size limit.")
#tf.app.flags.DEFINE_integer("max_seq_len", 200, "Maximum sequence length.")
tf.app.flags.DEFINE_integer("max_seq_len", 100, "Maximum sequence length.")
tf.app.flags.DEFINE_string("data_dir", "./", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "./", "Training directory.")
tf.app.flags.DEFINE_string("tokenizer", "CHAR", "BPE / CHAR / WORD.")
tf.app.flags.DEFINE_string("optimizer", "adam", "adam / sgd")
tf.app.flags.DEFINE_integer("print_every", 1, "How many iterations to do per print.")
FLAGS = tf.app.flags.FLAGS

def create_model(session, vocab_size, forward_only):
  model = NLCModel(
      vocab_size, FLAGS.size, FLAGS.num_layers, FLAGS.max_gradient_norm, FLAGS.batch_size,
      FLAGS.learning_rate, FLAGS.learning_rate_decay_factor, FLAGS.dropout,
      forward_only=forward_only, optimizer=FLAGS.optimizer)
  ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
  if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
    logging.info("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(session, ckpt.model_checkpoint_path)
  else:
    logging.info("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
    logging.info('Num params: %d' % sum(v.get_shape().num_elements() for v in tf.trainable_variables()))
  return model


def validate(model, sess, x_dev, y_dev):
  valid_costs, valid_lengths = [], []
  for source_tokens, source_mask, target_tokens, target_mask in pair_iter(x_dev, y_dev, FLAGS.batch_size, FLAGS.num_layers):
    cost = model.test(sess, source_tokens, source_mask, target_tokens, target_mask)
    valid_costs.append(cost * target_mask.shape[1])
    valid_lengths.append(np.sum(target_mask[1:, :]))
  valid_cost = sum(valid_costs) / float(sum(valid_lengths))
  return valid_cost


def train():
  """Train a translation model using NLC data."""
  # Prepare NLC data.
  logging.info("Preparing NLC data in %s" % FLAGS.data_dir)

  x_train, y_train, x_dev, y_dev, vocab_path = prepare_nlc_data(
    FLAGS.data_dir + '/' + FLAGS.tokenizer.lower(), FLAGS.max_vocab_size,
    tokenizer=get_tokenizer(FLAGS))
  vocab, _ = initialize_vocabulary(vocab_path)
  vocab_size = len(vocab)
  logging.info("Vocabulary size: %d" % vocab_size)

  file_handler = logging.FileHandler("{0}/log.txt".format(FLAGS.train_dir))
  logging.getLogger().addHandler(file_handler)

  print(vars(FLAGS))
  with open(os.path.join(FLAGS.train_dir, "flags.json"), 'w') as fout:
    json.dump(FLAGS.__flags, fout)

  with tf.Session() as sess:
    logging.info("Creating %d layers of %d units." % (FLAGS.num_layers, FLAGS.size))
    model = create_model(sess, vocab_size, False)

    logging.info('Initial validation cost: %f' % validate(model, sess, x_dev, y_dev))

    if False:
      tic = time.time()
      params = tf.trainable_variables()
      num_params = sum(map(lambda t: np.prod(tf.shape(t.value()).eval()), params))
      toc = time.time()
      print ("Number of params: %d (retreival took %f secs)" % (num_params, toc - tic))

    epoch = 0
    best_epoch = 0
    previous_losses = []
    exp_cost = None
    exp_length = None
    exp_norm = None
    total_iters = 0
    start_time = time.time()
    while (FLAGS.epochs == 0 or epoch < FLAGS.epochs):
      epoch += 1
      current_step = 0

      ## Train
      epoch_tic = time.time()
      for source_tokens, source_mask, target_tokens, target_mask in pair_iter(x_train, y_train, FLAGS.batch_size, FLAGS.num_layers):
        # Get a batch and make a step.
        tic = time.time()

        grad_norm, cost, param_norm = model.train(sess, source_tokens, source_mask, target_tokens, target_mask)

        toc = time.time()
        iter_time = toc - tic
        total_iters += np.sum(target_mask)
        tps = total_iters / (time.time() - start_time)
        current_step += 1

        lengths = np.sum(target_mask, axis=0)
        mean_length = np.mean(lengths)
        std_length = np.std(lengths)

        if not exp_cost:
          exp_cost = cost
          exp_length = mean_length
          exp_norm = grad_norm
        else:
          exp_cost = 0.99*exp_cost + 0.01*cost
          exp_length = 0.99*exp_length + 0.01*mean_length
          exp_norm = 0.99*exp_norm + 0.01*grad_norm

        cost = cost / mean_length

        if current_step % FLAGS.print_every == 0:
          logging.info('epoch %d, iter %d, cost %f, exp_cost %f, grad norm %f, param norm %f, tps %f, length mean/std %f/%f' %
                (epoch, current_step, cost, exp_cost / exp_length, grad_norm, param_norm, tps, mean_length, std_length))
      epoch_toc = time.time()

      ## Checkpoint
      checkpoint_path = os.path.join(FLAGS.train_dir, "best.ckpt")

      ## Validate
      valid_cost = validate(model, sess, x_dev, y_dev)

      logging.info("Epoch %d Validation cost: %f time: %f" % (epoch, valid_cost, epoch_toc - epoch_tic))

      if len(previous_losses) > 2 and valid_cost > previous_losses[-1]:
        logging.info("Annealing learning rate by %f" % FLAGS.learning_rate_decay_factor)
        sess.run(model.learning_rate_decay_op)
        model.saver.restore(sess, checkpoint_path + ("-%d" % best_epoch))
      else:
        previous_losses.append(valid_cost)
        best_epoch = epoch
        model.saver.save(sess, checkpoint_path, global_step=epoch)
      sys.stdout.flush()

train()